# Twitter Trust Correlation

In [0]:
import pandas as pd
import numpy as np
import pickle as pk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!wget http://muftayebaly.com/dropbox/tweetsCID.csv

--2020-04-09 21:56:51--  http://muftayebaly.com/dropbox/tweetsCID.csv
Resolving muftayebaly.com (muftayebaly.com)... 107.180.20.85
Connecting to muftayebaly.com (muftayebaly.com)|107.180.20.85|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54234968 (52M) [text/csv]
Saving to: ‘tweetsCID.csv’

tweetsCID.csv       100%[===================>]  51.72M  15.4MB/s    in 3.9s    

2020-04-09 21:56:55 (13.2 MB/s) - ‘tweetsCID.csv’ saved [54234968/54234968]



In [4]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_articles.csv

--2020-04-09 21:57:39--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9186850 (8.8M) [text/plain]
Saving to: ‘knight_data_articles.csv’

knight_data_article 100%[===================>]   8.76M  --.-KB/s    in 0.05s   

2020-04-09 21:57:39 (181 MB/s) - ‘knight_data_articles.csv’ saved [9186850/9186850]



In [5]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_urls.csv

--2020-04-09 21:57:43--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_urls.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9199526 (8.8M) [text/plain]
Saving to: ‘knight_data_urls.csv’

knight_data_urls.cs 100%[===================>]   8.77M  --.-KB/s    in 0.05s   

2020-04-09 21:57:43 (192 MB/s) - ‘knight_data_urls.csv’ saved [9199526/9199526]



In [6]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_ratings.csv

--2020-04-09 21:57:46--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2344804 (2.2M) [text/plain]
Saving to: ‘knight_data_ratings.csv’

knight_data_ratings 100%[===================>]   2.24M  --.-KB/s    in 0.03s   

2020-04-09 21:57:46 (71.6 MB/s) - ‘knight_data_ratings.csv’ saved [2344804/2344804]



In [7]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/processed/article_topics_6.csv

--2020-04-09 21:57:49--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/processed/article_topics_6.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6035965 (5.8M) [text/plain]
Saving to: ‘article_topics_6.csv’

article_topics_6.cs 100%[===================>]   5.76M  --.-KB/s    in 0.1s    

2020-04-09 21:57:50 (46.1 MB/s) - ‘article_topics_6.csv’ saved [6035965/6035965]



In [22]:
raw_topics = pd.read_csv("article_topics_6.csv")
topics = raw_topics.set_index(['content_id'])
topics.head()

,Unnamed: 0,content_body_clean,content_title_clean,topic,topic_id1,topic_p1,topic_id2,topic_p2,topic_dist
content_id,,,,,,,,,
60,0,republican congress make break moment everyone...,tax reform failure act could mean minority sta...,Policy,0,0.548654,4.0,0.398622,"[(0, 0.54865676), (1, 0.035524607), (2, 0.0146..."
61,1,constitution authorize congress tax american p...,tax reform america need tax code good country ...,Policy,0,0.832200,3.0,0.098822,"[(0, 0.83220094), (3, 0.098824225), (4, 0.0664..."
66,2,longmothballed unfinished casinohotel la vega ...,longmothballed la vega strip casinohotel sell,Economy,2,0.677059,3.0,0.147280,"[(1, 0.09982404), (2, 0.6770535), (3, 0.147273..."
80,3,tax foreign earnings already accumulate overse...,trump tax plan may free corporate dollar,Policy,0,0.994644,NaN,NaN,"[(0, 0.9946441)]"
82,4,hurricane harvey already take enormous human t...,hurricane harvey humanitarian disaster also se...,Public Health,6,0.759787,2.0,0.237297,"[(2, 0.23729898), (6, 0.7597849)]"


In [0]:
raw_data = pd.read_csv("knight_data_articles.csv")
#raw_data.head()

In [9]:
articles = raw_data.set_index(['content_id'])
articles

,content_body_clean,content_title_clean,label,economy,topic2,science,mention_Trump,mention_Clinton,mention_Democrat,mention_Republican,mention_GOP,mention_Administration,mention_political,mention_voters,mention_campaign,mention_climate,mention_abortion,mention_immigration,mention_trade,mention_immigrants,mention_border,mention_terrorist,mention_attack,mention_NRA,mention_sanctuary,mention_socialist,mention_fascist,mention_communist,mention_nationalist,mention_2A,content_source_desc
content_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
60,Republicans in Congress are at a make or break...,Tax reform now -- failure to act could mean mi...,9,1,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fox News
61,The Constitution authorizes Congress to tax Am...,Tax reform -- America needs a tax code that's ...,9,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fox News
66,"A long-mothballed, unfinished casino-hotel on ...",Long-mothballed Las Vegas Strip casino-hotel s...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fox News
80,If taxing foreign earnings that have already a...,"Trump Tax Plan May Free Up Corporate Dollars, ...",3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,The New York Times
82,Hurricane Harvey has already taken an enormous...,Hurricane Harvey is a humanitarian disaster. I...,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Vox
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3380,"In this Oct. 17, 2017 file photo, Venezuela's...",Investors leave Venezuela meeting with no clea...,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Associated Press News
3381,A man watches stock prices boards at a brokera...,Global stocks turn lower after recent record h...,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Associated Press News
3382,The draining of a massive aquifer that underl...,Rural areas at risk as water levels drop in ma...,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Associated Press News


In [0]:
raw_urls = pd.read_csv("knight_data_urls.csv")
#raw_urls.head()

In [12]:
raw_tweets = pd.read_csv("tweetsCID.csv")
raw_tweets

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id
0,851485433003421701,#Trump’s border wall would be an ecological an...,613521891,2017-04-10T17:22:01.000Z,0,0,https://www.vox.com/energy-and-environment/201...,2736
1,851514117374398464,#Trump’s border wall would be an ecological an...,222481411,2017-04-10T19:16:00.000Z,0,0,https://www.vox.com/energy-and-environment/201...,2736
2,855009400670617600,The research and experts are clear: Tougher bo...,2347049341,2017-04-20T10:45:01.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
3,855009683542728704,Hmmmmmm https://t.co/y3fNx7KuJT,243580387,2017-04-20T10:46:08.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
4,855011873183342592,(psst! It was never about drugs. It's always b...,17889654,2017-04-20T10:54:50.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
...,...,...,...,...,...,...,...,...
201868,930566176907853824,Big increase in investment hastening arrival o...,454368263,2017-11-14T22:40:40.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351
201869,930571261775876097,As readers of https://t.co/ajmt36bUQU already ...,61681958,2017-11-14T23:00:52.000Z,0,0,https://tytnetwork.com/Investigates/; https://...,3351
201870,930573435343908864,We’ve been reporting on what FedEx and UPS are...,905632398511046656,2017-11-14T23:09:30.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351
201871,930582220800233473,Self-Driving Trucks May Be Closer Than They Ap...,831215040,2017-11-14T23:44:25.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351


In [13]:
raw_tweets.describe()

,tweet_id,user_id,no_favs,no_retweets
count,2.018730e+05,2.018730e+05,201873.0,201873.0
mean,9.207283e+17,1.332248e+17,0.0,0.0
std,7.496690e+15,3.013590e+17,0.0,0.0
min,8.514854e+17,5.700000e+01,0.0,0.0
25%,9.174132e+17,5.645870e+07,0.0,0.0
50%,9.218926e+17,4.205027e+08,0.0,0.0
75%,9.260537e+17,2.853383e+09,0.0,0.0
max,9.305854e+17,9.301665e+17,0.0,0.0


In [14]:
raw_ratings = pd.read_csv("knight_data_ratings.csv")
raw_ratings.head()

,external_user_id,content_id,rating_scale_response,sampdate,blind
0,047641_S02_01,60,4.0,20170901,0
1,4205985366_78419_01,60,5.0,20170901,1
2,4205985366_78419_01,61,4.5,20170901,1
3,4205985366_78419_01,66,3.5,20170901,1
4,4205985366_78419_01,80,4.0,20170901,1


In [0]:
import os
import shutil

## Correlation of \# of Tweets and Trust

In [16]:
blind_ratings = raw_ratings[raw_ratings['blind']==1].groupby(['content_id']).mean()
blind_ratings['content_id1'] = blind_ratings.index
blind_ratings

,rating_scale_response,sampdate,blind,content_id1
content_id,,,,
60,5.000,20170901,1,60
61,4.500,20170901,1,61
66,3.500,20170901,1,66
80,3.750,20170901,1,80
82,3.500,20170901,1,82
...,...,...,...,...
3375,2.500,20170901,1,3375
3378,4.625,20170901,1,3378
3380,3.500,20170901,1,3380


In [17]:
num_tweets = raw_tweets.groupby(['content_id']).count()
num_tweets['content_id1'] = num_tweets.index
num_tweets

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id1
content_id,,,,,,,,
101,33,33,33,33,33,33,33,101
1019,14,14,14,14,14,14,14,1019
102,20,20,20,20,20,20,20,102
103,18,18,18,18,18,18,18,103
1060,2,2,2,2,2,2,2,1060
...,...,...,...,...,...,...,...,...
984,14,14,14,14,14,14,14,984
985,273,273,273,273,273,273,273,985
990,8,8,8,8,8,8,8,990


In [23]:
corr_df = pd.DataFrame(columns=['content_id','num_tweets','trust_mean','source','topic'])

for index, row in num_tweets.iterrows():
  try:
    corr_df = corr_df.append({'content_id':index, 'num_tweets':row['tweet_id'], 'trust_mean':blind_ratings['rating_scale_response'][int(index)], 
                              'source':articles['content_source_desc'][int(index)], 'topic':topics['topic'][int(index)]}, ignore_index=True)
  except Exception:
    pass

corr_df

,content_id,num_tweets,trust_mean,source,topic
0,101,33,4.000000,Media Matters,Public Health
1,1019,14,1.000000,Media Matters,Economy
2,102,20,0.500000,Media Matters,Public Health
3,103,18,0.500000,Media Matters,Politics
4,1060,2,4.000000,Associated Press News,Public Health
...,...,...,...,...,...
1250,969,13,2.000000,Media Matters,Politics
1251,984,14,4.166667,Fox News,Policy
1252,985,273,2.750000,The New York Times,Public Health
1253,990,8,2.800000,Vox,Technology


In [0]:
from scipy.stats.stats import pearsonr

In [0]:
pearsonr(corr_df['num_tweets'],corr_df['trust_mean'])[0]

0.050122302409525775

In [0]:
np.corrcoef(corr_df['num_tweets'].values.tolist(),corr_df['trust_mean'].values.tolist())[0, 1]

0.05012230240952584

In [0]:
corr_df1 = corr_df.astype({'num_tweets': 'int32'})

In [25]:
corr_df1

,content_id,num_tweets,trust_mean,source,topic
0,101,33,4.000000,Media Matters,Public Health
1,1019,14,1.000000,Media Matters,Economy
2,102,20,0.500000,Media Matters,Public Health
3,103,18,0.500000,Media Matters,Politics
4,1060,2,4.000000,Associated Press News,Public Health
...,...,...,...,...,...
1250,969,13,2.000000,Media Matters,Politics
1251,984,14,4.166667,Fox News,Policy
1252,985,273,2.750000,The New York Times,Public Health
1253,990,8,2.800000,Vox,Technology


In [26]:
corr_df1.groupby(['source']).mean()

,num_tweets,trust_mean
source,,
100 Percent Fed Up,38.588235,2.222855
Associated Press News,4.295238,3.559105
Breitbart,130.113402,2.510586
Fox News,99.490196,3.264439
Media Matters,24.098039,2.572640
The New York Times,294.234177,3.555893
Vox,102.771552,3.373520


In [27]:
corr_df1.groupby(['topic']).mean()

,num_tweets,trust_mean
topic,,
Economy,87.508197,3.173621
Policy,183.082902,3.177887
Politics,123.334184,2.992726
Public Health,149.663830,3.543269
Scandal,166.871698,2.980684
Technology,42.541284,3.255277


In [40]:
print('100 Percent Fed Up:', pearsonr(corr_df[corr_df['source']=='100 Percent Fed Up'].num_tweets,
                                     corr_df[corr_df['source']=='100 Percent Fed Up'].trust_mean
                                     )[0])
print('Associated Press News:', pearsonr(corr_df[corr_df['source']=='Associated Press News'].num_tweets,
                                     corr_df[corr_df['source']=='Associated Press News'].trust_mean
                                     )[0])
print('Breitbart:', pearsonr(corr_df[corr_df['source']=='Breitbart'].num_tweets,
                                     corr_df[corr_df['source']=='Breitbart'].trust_mean
                                     )[0])
print('Fox News:', pearsonr(corr_df[corr_df['source']=='Fox News'].num_tweets,
                                     corr_df[corr_df['source']=='Fox News'].trust_mean
                                     )[0])
print('Media Matters:', pearsonr(corr_df[corr_df['source']=='Media Matters'].num_tweets,
                                     corr_df[corr_df['source']=='Media Matters'].trust_mean
                                     )[0])
print('The New York Times:', pearsonr(corr_df[corr_df['source']=='The New York Times'].num_tweets,
                                     corr_df[corr_df['source']=='The New York Times'].trust_mean
                                     )[0])
print('Vox:', pearsonr(corr_df[corr_df['source']=='Vox'].num_tweets,
                                     corr_df[corr_df['source']=='Vox'].trust_mean
                                     )[0])

100 Percent Fed Up: -0.029205437098217952
Associated Press News: -0.13419177533207938
Breitbart: -0.052556843882955256
Fox News: -0.27429417636187603
Media Matters: -0.05603416234001038
The New York Times: 0.08016191474413324
Vox: 0.03539024107968908
